**⚠️  This notebook is de-activated in CI – awaiting raw data from MIT (expected file: paircorr_raw_MIT.csv).**

In [ ]:
import os, sys
if os.getenv("CI"):
    raise SystemExit("Notebook skipped in CI – data pending.")

In [ ]:
# --- 🚀 TORUS universal imports / helpers ---
from torus_bootstrap import *  # universal imports & seed